In [1]:
using JLD2
using FileIO
using PyPlot
using Statistics
using StatsBase 
using Distributions
using LinearAlgebra
using HDF5
using KernelDensity
using IntervalSets
using Random, ArraysOfArrays
using ValueShapes
using Measurements
using BAT
using DSP
using ForwardDiff

In [2]:
include("../utill.jl")

plot_projections (generic function with 1 method)

In [3]:
FPATH = "../../data/sampling_results/Benchmark-5/"

function marg_mean(name)
    smpl_tmp = bat_read(FPATH*name)
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp = unshaped(mean(smpl_tmp))
    return smpl_tmp
end

function marg_mean_shaped(name)
    smpl_tmp = bat_read(FPATH*name)
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp = mean(smpl_tmp)
    return smpl_tmp
end

function marg_std(name)
    smpl_tmp = bat_read(FPATH*name)
    smpl_tmp = varshape(prior).(smpl_tmp)
    smpl_tmp = unshaped(std(smpl_tmp))
    return smpl_tmp
end

function cov_matr(name)
    smpl_tmp = bat_read(FPATH*name)
    smp_fl, w = flatview(unshaped.(smpl_tmp.v)), smpl_tmp.weight
    smp_fl = smp_fl ./ std(smp_fl, FrequencyWeights(w), 2)
    smp_fl = smp_fl .- mean(smp_fl, FrequencyWeights(w), dims=2)
    cov_m = cov(smp_fl, FrequencyWeights(w), 2)
    return cov_m
end

cov_matr (generic function with 1 method)

In [4]:
β1 = 0.015
β2 = 0.0077 # (0.8/30/sqrt(12))
β3 = 0.0058 # (0.8/40/sqrt(12))

# flat prior for the pixel size
prior = NamedTupleDist(
        tr_size = [truncated(Normal(0.2, 0.04), 0.06, 0.19), truncated(Normal(0.2, 0.04), 0.06, 0.19)],
        tr_size_2 = [truncated(Normal(0.2, 0.04), 0.06, 0.19), truncated(Normal(0.2, 0.04), 0.06, 0.19)],
        ang_spr = [truncated(Normal(4.0, 2.0), 4.0, 7.0), truncated(Normal(4.0, 2.0), 4.0, 7.0)],
        ang_spr_2 = [truncated(Normal(4.0, 2.0), 1.0, 4.0), truncated(Normal(4.0, 2.0), 1.0, 4.0)],
        mixt_pow =  0.50 .. 1.0 ,
        waist = [truncated(Normal(2.9, 0.03), 2.65, 3.5)],
        waist_2 = [truncated(Normal(2.9, 0.03), 2.65, 3.5)], # 11
        algmx = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
        algmy = [23.0 .. 48, 23.0 .. 48.0, 10.0 .. 30.0, 23.0 .. 48.0],
        cam4_ped = 4.0 .. 40.0,
        cam4_light_fluct = 1.0 .. 3.0,
        cam4_light_amp = 1.6 .. 9.9, 
        resx = [1, 1, 1], # 23, 24, 25, 
        resy = [1, 1, 1], # 26,27, 28, 
        cam4_resx = truncated(Normal(3, 1.5), 0, Inf),
        cam4_resy = truncated(Normal(3, 1.5), 0, Inf), 
        psx = [27.1, 21.6, 114.0], # 31, 32, 33
        psy = [30.5, 23.4, 125.0], # 34, 35, 36
        cam4_psx = 121.8, # 37
        cam4_psy = 120.0, # 38
        light_amp  = [1.0 .. 13.0 , 1.0 .. 17.0, 1.0 .. 5.0], # 1.0 .. 5.0
        s_cam = [0.0, 1.478, 15.026, 23.1150],
    )


labels = [L"\sigma_x", L"\sigma_y", L"\sigma_x", L"\sigma_y", L"\sigma'_x", L"\sigma'_y", L"\sigma'_x", L"\sigma'_y", L"\alpha", L"s", L"s", 
    L"\mu_x", L"\mu_x", L"\mu_x", L"\mu_x", L"\mu_y", L"\mu_y", L"\mu_y", L"\mu_y", 
    "ped", "lf", "a", "res x", "res y", "la", "la", "la"];

In [6]:
smpl_tmp = bat_read(FPATH*"hc-2.hdf5")
smpl_tmp = varshape(prior).(smpl_tmp)
@show mode(smpl_tmp)[1]

(mode(smpl_tmp))[1] = (tr_size = [0.09022016247626924, 0.13278027377370621], tr_size_2 = [0.13087542193714233, 0.15158061149510935], ang_spr = [6.227488514589307, 5.755259786375036], ang_spr_2 = [2.6057915179686617, 2.397589287133589], mixt_pow = 0.666006673110822, waist = [3.1425423563421417], waist_2 = [2.8738232953734717], algmx = [31.741703234746318, 32.22277650028469, 21.291812547178893, 35.2540699308112], algmy = [36.930260443041675, 37.40757355716955, 20.25845816811686, 33.90824607487838], cam4_ped = 27.57169693775633, cam4_light_fluct = 1.365633716171961, cam4_light_amp = 8.846914204881422, resx = [1, 1, 1], resy = [1, 1, 1], cam4_resx = 4.827560766404559, cam4_resy = 4.7854307298949275, psx = [27.1, 21.6, 114.0], psy = [30.5, 23.4, 125.0], cam4_psx = 121.8, cam4_psy = 120.0, light_amp = [8.532172007081927, 11.430580961050259, 2.6946677414758087], s_cam = [0.0, 1.478, 15.026, 23.115])


(tr_size = [0.09022016247626924, 0.13278027377370621], tr_size_2 = [0.13087542193714233, 0.15158061149510935], ang_spr = [6.227488514589307, 5.755259786375036], ang_spr_2 = [2.6057915179686617, 2.397589287133589], mixt_pow = 0.666006673110822, waist = [3.1425423563421417], waist_2 = [2.8738232953734717], algmx = [31.741703234746318, 32.22277650028469, 21.291812547178893, 35.2540699308112], algmy = [36.930260443041675, 37.40757355716955, 20.25845816811686, 33.90824607487838], cam4_ped = 27.57169693775633, cam4_light_fluct = 1.365633716171961, cam4_light_amp = 8.846914204881422, resx = [1, 1, 1], resy = [1, 1, 1], cam4_resx = 4.827560766404559, cam4_resy = 4.7854307298949275, psx = [27.1, 21.6, 114.0], psy = [30.5, 23.4, 125.0], cam4_psx = 121.8, cam4_psy = 120.0, light_amp = [8.532172007081927, 11.430580961050259, 2.6946677414758087], s_cam = [0.0, 1.478, 15.026, 23.115])

In [7]:
filenames = readdir("../../data/sampling_results/Benchmark-5/")
images = load("../../data/experiment/dataset_2/m2/images-satur.jld2")
hc_mask = [i[1:3] == "hc-" for i in filenames]
hc_fnames = filenames[hc_mask]
hc_eind = parse.(Int64, [i[4:end-5] for i in hc_fnames])
hc_charge = images["charge"][hc_eind]
hc_runind = images["ev_ind"][hc_eind];

In [8]:
filenames = readdir("../../data/sampling_results/Benchmark-5/")
images = load("../../data/experiment/dataset_2/m1/images-satur.jld2")
lc_mask = [i[1:3] == "lc-" for i in filenames]
lc_fnames = filenames[lc_mask]
lc_eind = parse.(Int64, [i[4:end-5] for i in lc_fnames])
lc_charge = images["charge"][lc_eind]
lc_runind = images["ev_ind"][lc_eind];

In [9]:
fnames = [lc_fnames; hc_fnames]
charges = [lc_charge; hc_charge]
runind = [lc_runind ; hc_runind]
charges_n = charges ./ maximum(charges)
runind_n = runind ./ maximum(runind);

In [10]:
msklc = 1:length(lc_fnames)
mskhc = length(lc_fnames)+1:length(lc_fnames)+length(hc_fnames);

In [11]:
charges_n[sortperm(charges_n)] .= 1:length(charges_n)
charges_n = charges_n ./ maximum(charges_n);

In [ ]:
# mean_params =  [marg_mean(i) for i in fnames[mskhc]]
# mean_matr = hcat(mean_params...);

In [12]:
mean_params =  [marg_mean_shaped(i) for i in fnames];

In [27]:
# using JLD2

# tr_1_x = [i.tr_size[1] for i in mean_params]
# tr_1_y = [i.tr_size[2] for i in mean_params]
# tr_2_x = [i.tr_size_2[1] for i in mean_params]
# tr_2_y = [i.tr_size_2[2] for i in mean_params]
# as_1_x = [i.ang_spr[1] for i in mean_params]
# as_1_y = [i.ang_spr[2] for i in mean_params]
# as_2_x = [i.ang_spr_2[1] for i in mean_params]
# as_2_y = [i.ang_spr_2[2] for i in mean_params]
# w_1 = [i.waist[1] for i in mean_params]
# w_2 = [i.waist_2[1] for i in mean_params]

# summary_data = (
#     tr_1_x=tr_1_x, 
#     tr_1_y=tr_1_y, 
#     tr_2_x=tr_2_x, 
#     tr_2_y=tr_2_y, 
#     as_1_x=as_1_x, 
#     as_1_y=as_1_y, 
#     as_2_x=as_2_x, 
#     as_2_y=as_2_y, 
#     w_1=w_1, 
#     w_2=w_2, 
#     μ_x = data.x_event, 
#     μ_y = data.y_event,
#     charge = charges,
#     run_id = runind
# );

# @save "../../data/sampling_results/Benchmark-5/alignment/mean-params-mcmc.jld" summary_data

In [23]:
x_event = Array{Float64,1}[]
y_event = Array{Float64,1}[]

n_events = 5 #length(mskhc)
n_cameras = 4

for i in eachindex(mean_params) #[mskhc]
    push!(x_event, mean_params[i].algmx .* [mean_params[i].psx ; mean_params[i].cam4_psx] .* 10^-3)
    push!(y_event, mean_params[i].algmy .* [mean_params[i].psy ; mean_params[i].cam4_psy] .* 10^-3)
end

data = (x_event = x_event, y_event = y_event, s_cam = mean_params[1].s_cam[1:4]);

In [24]:
data.x_event

66-element Array{Array{Float64,1},1}:
 [0.9082688603006076, 0.7373587740758002, 2.471069430467353, 4.340306979548877]
 [0.9791934233017083, 0.8028534071066775, 2.4116016496464585, 4.39326021558258]
 [0.8896746934461729, 0.7165939842039926, 2.3770187418827318, 4.428408711620615]
 [0.9048437809761471, 0.7411438732658683, 2.4326259313675007, 4.324289193397275]
 [0.9015534297206127, 0.7366067643303007, 2.4169165155408843, 4.387306854178396]
 [0.9552558022440436, 0.7856064521869628, 2.4649984739032216, 4.319950560416011]
 [0.9816945344800468, 0.8129904768856774, 2.4819890671440112, 4.278147579844197]
 [0.9714995964981143, 0.8031554386043136, 2.45130819599804, 4.327435954134791]
 [0.9738391566217356, 0.8036159599535283, 2.4784325781451506, 4.279547198271023]
 [0.8764057869276578, 0.7099204921465885, 2.4016152665177617, 4.360528574708276]
 [0.9015676452196713, 0.7243741007447652, 2.3953287157267975, 4.389414076184244]
 [0.9020488277779724, 0.7261307542548107, 2.3859462401638707, 4.38246184245

In [18]:
prior_ang = NamedTupleDist(
    θ = [10^-20 .. 10^-4 for i in 1:n_events],
    α = [0 .. 2*pi  for i in 1:n_events],
    x_center = [-5 .. 5 for i in 1:n_events],
    y_center = [-5 .. 5 for i in 1:n_events],
    x0 = [-5 .. 5 for i in 1:n_cameras],
    y0 = [-5 .. 5 for i in 1:n_cameras],
    σ_x = [10^-11 .. 0.2 for i in 1:n_cameras],
    σ_y = [10^-11 .. 0.2 for i in 1:n_cameras],
);

In [ ]:
function log_lik(; data = data, nevents = n_events)
    
    return params -> begin   
        ll = 0.0
        
        for i in 1:nevents
            x_expected = cos(params.α[i]).*tan(params.θ[i]).*(data.s_cam - 2.9).*10^3
            y_expected = sin(params.α[i]).*tan(params.θ[i]).*(data.s_cam - 2.9).*10^3
            
            ll += sum(logpdf.(Normal.(data.x_event[i][1] .+ params.x0 .+ x_expected, params.σ_x), data.x_event[i][2:4]))
            ll += sum(logpdf.(Normal.(data.y_event[i][1] .+ params.y0  .+ y_expected, params.σ_y), data.y_event[i][2:4]))
        end
        return LogDVal(ll)
    end
        
end

log_likelihood = log_lik()

In [ ]:
posterior = PosteriorDensity(log_likelihood, prior_ang);

In [ ]:
posterior_is = bat_transform(PriorToGaussian(), posterior, PriorSubstitution()).result

iters = 60000
iters_warmup = 1200
chains = 5

metric = BAT.DiagEuclideanMetric()
integrator = BAT.LeapfrogIntegrator(0.0)

proposal = BAT.NUTS(:MultinomialTS, :ClassicNoUTurn)
# proposal = BAT.FixedTrajectoryLength()

adaptor = BAT.StanHMCAdaptor(0.8, iters_warmup)
# adaptor = BAT.StepSizeAdaptor()

ahmc_sampler = HamiltonianMC(metric, ForwardDiff, integrator, proposal, adaptor);

In [ ]:
@time samples_is = bat_sample(posterior_is, iters, MCMCSampling(sampler = ahmc_sampler, nchains = chains));

In [ ]:
samples = samples_is.result;

In [ ]:
trafo_is = trafoof(posterior_is.likelihood)
samples = inv(trafo_is).(samples);

In [ ]:
# BAT.bat_write("../../data/sampling_results/Benchmark-5/alignment/hc-samples.hdf5", unshaped.(samples))

In [ ]:
# @save "../../data/sampling_results/Benchmark-5/alignment/hc-data.hdf5" data

In [ ]:
# samples = samples_tot.result;

In [ ]:
# minimum(bat_eff_sample_size(unshaped.(samples)).result)

In [ ]:
samples = bat_read("../../data/sampling_results/Benchmark-5/alignment/hc-samples.hdf5")
samples = varshape(prior_ang).(samples);

In [ ]:
sz = size(flatview(unshaped.(samples.v)))

In [ ]:
plot(flatview(unshaped.(samples.v))[3,:])

In [ ]:
function cov_matr(smpl_tmp)
    smp_fl, w = flatview(unshaped.(smpl_tmp.v)), smpl_tmp.weight
    smp_fl = smp_fl ./ std(smp_fl, FrequencyWeights(w), 2)
    smp_fl = smp_fl .- mean(smp_fl, FrequencyWeights(w), dims=2)
    cov_m = cov(smp_fl, FrequencyWeights(w), 2)
    return cov_m
end

In [ ]:
covm = cov_matr(samples);

fig, (ax, ax2) = plt.subplots(1,2, figsize=(7,7), gridspec_kw=Dict("width_ratios"=>[15,1]))

max = maximum(abs.(covm))

cb = ax.imshow(covm,  origin="upper", cmap="seismic", vmin=-max, vmax=max, aspect="equal") 
fig.colorbar(cb, spacing="proportional", label="Covariance", cax=ax2)

# ax.set_yticks(0:26)
# ax.set_yticklabels(labels)

# ax.set_xticks(0:26)
# ax.set_xticklabels(labels, rotation = 90);

In [ ]:
x = flatview(unshaped.(samples.v))'

plt.imshow(log.(abs.(cov(flatview(unshaped.(samples.v))'))), cmap="Blues")

In [ ]:
#  27.1, 21.6, 114.0, 30.5, 23.4, 125.0, 121.8, 120.0

In [ ]:
param_mode = mode(samples)

In [ ]:
bat_marginalmode(samples).result

In [ ]:
mean(samples)[1]

In [ ]:
std(samples)[1]

In [ ]:
#  findmode_result = bat_findmode(posterior, MaxDensityLBFGS(init = InitFromTarget()), )

findmode_result = bat_findmode(
    posterior, 
    MaxDensityLBFGS(init = ExplicitInit([param_mode],)), 
)

fit_par_values = findmode_result.result

In [ ]:
# @save "../../data/sampling_results/Benchmark-5/alignment/hc-data.jld2" data fit_par_values

In [ ]:
dims = 9:14
# dims = [1,2,3,4,5,]
names = dims
levels = [0.01, 0.55, 0.9, 0.95,  1,]
# levels = [0.001, 0.98, 1,]
figsize = (10,10)
n_bins = 30

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)

In [ ]:
dims = 1:n_events
# dims = [1,2,3,4,5,]
names = dims
levels = [0.01, 0.55, 0.9, 0.95,  1,]
# levels = [0.001, 0.98, 1,]
figsize = (10,10)
n_bins = 30

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)

In [ ]:
dims = n_events+1:2*n_events
# dims = [1,2,3,4,5,]
names = dims
levels = [0.01, 0.55, 0.9, 0.95,  1,]
levels = [0.001, 0.98, 1,]
figsize = (10,10)
n_bins = 25

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)

In [ ]:
dims = 50:59
# dims = [1,2,3,4,5,]
names = dims
levels = [0.01, 0.55, 0.9, 0.95,  1,]
# levels = [0.001, 0.98, 1,]
figsize = (10,10)
n_bins = 25

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)

In [ ]:
dims = collect(sz[1]-11:sz[1]-6)
names = dims
levels = [0.01, 0.55, 0.9, 0.95,  1,]
levels = [0.001, 0.95, 1,]
figsize = (10,10)
n_bins = 25

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)

In [ ]:
dims = collect(sz[1]-5:sz[1])
names = dims
levels = [0.01, 0.55, 0.9, 0.95,  1,]
levels = [0.001, 0.98, 1,]
figsize = (10,10)
n_bins = 25

corner_plots(samples, dims, names, levels_quantiles = levels, figsize=figsize, N_bins=n_bins)